In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import re

In [0]:
# Read recipe inputs
dictionaries_of_aviation = dataiku.Dataset("dictionaries_of_aviation")
dictionaries_of_aviation_df = dictionaries_of_aviation.get_dataframe()

In [0]:
dictionaries_of_aviation_df

In [0]:
dictionary = dictionaries_of_aviation_df["content"][2]

In [0]:
dictionary

In [0]:
clean_dictionary = dictionary.replace("-\n","")
clean_dictionary = dictionary.replace(" \n"," ")

#clean_dictionary = re.sub(r'\s*\n\s*', ' ', clean_dictionary)
clean_dictionary = re.sub(r'([a-zA-Z]|[0-9])\n([a-zA-Z]|[0-9])', r'\1 \2', clean_dictionary)
clean_dictionary = re.sub(" \x02 ", ' ', clean_dictionary)

In [0]:
clean_dictionary

In [0]:





# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

dale_crane_dic_df = dictionaries_of_aviation_df # For this sample code, simply copy input to output




In [0]:
# Write recipe outputs
dale_crane_dic = dataiku.Dataset("dale_crane_dic")
dale_crane_dic.write_with_schema(dale_crane_dic_df)